# Road Following - Live demo

In this notebook, we will use model we trained to move jetBot smoothly on track. 

### Load Trained Model

In [3]:
import torchvision
import torch
from yolov5.object_detect import run as object_detect_inference
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.torch_utils import select_device, smart_inference_mode
from yolov5.utils.general import scale_coords, non_max_suppression

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

labels_to_names = {0:'Stop_Sign'}
ob_device = select_device('cuda:0')
ob_weights = 'best.pt' # best.pt 경로 설정
ob_model = DetectMultiBackend(ob_weights, device=ob_device)

YOLOv5 🚀 2022-9-29 Python-3.6.9 torch-1.7.0 CUDA:0 (NVIDIA Tegra X1, 3964MiB)

YOLOv5 🚀 2022-9-29 Python-3.6.9 torch-1.7.0 CUDA:0 (NVIDIA Tegra X1, 3964MiB)

Fusing layers... 
Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients


In [2]:
model.load_state_dict(torch.load('best_steering_model_xy_0920.pth')) # steering model명 기입

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

### Creating the Pre-Processing Function

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def ob_preprocess(image):
    image = np.array(image)
    image = torch.from_numpy(image).to(device)
    #image = transforms.functional.to_tensor(image).to(device)
    image = image.half() if ob_model.fp16 else image.float()  # uint8 to fp16/32
    image /= 255
    image = image.permute(2, 0, 1)
    transform = transforms.Resize((640, 640))
    image = transform(image)
    if len(image.shape) == 3:
        image = image[None]
    return image

### Camera Setting

In [5]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [ ]:
from jetbot import Robot
robot = Robot()

### Execution

In [22]:
import time
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0

    speed_offset = 0.34 # 기본 speed
    
    angle_diff_bias = 0.1
    angle = np.arctan2(x, y) # 핸들링 angle
    if abs(angle) + angle_diff_bias < abs(angle_last): # 이전 angle보다 현재 angle이 더 작을경우
        angle_temp = -1 * angle * 0.3
    else: # 
        angle_temp = angle #이전 angle 보다 현재 angle이 더 클 경우
    speed_value = speed_offset - abs(angle_temp) * 0.05 # 방향전환 상황에서는 속도를 낮춤
    
    if abs(angle_temp) < 0.05:
        angle_temp = 0
    
    angle_last = angle
    
    angle_temp *= 1.4 # 핸들링 배율

    # Yolo v5
    pred = ob_model(ob_preprocess(image), augment=False)
    
    # NMS
    pred = non_max_suppression(pred, conf_thres = 0.25, iou_thres = 0.45, classes = None, agnostic = False, max_det=1000)
    
    pred = pred[0] # => pred : bounding box 좌표(x1, y1, x2, y2), confidence score, class => total 6개의 값으로 나타남.

    bbox_threshold = 0 # 설정해야할 값(bbox가 얼마나 클 때 장애물로 인식할 것인지?)
    
    bbox_count = 0
    for *box, cf, cls in pred:
        cf = cf.item()
        cls = cls.item()

        p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
        
        bbox_area = (p2[0] - p1[0]) * (p2[1] - p1[1])
        if bbox_area > bbox_threshold:
            bbox_count = 1
#     print("object : {}, left_motor : {}, right_motor : {}".format(bbox_count, robot.left_motor.value, robot.right_motor.value))

    if bbox_count > 0:
      robot.left_motor.value = 0.0
      robot.right_motor.value = angle_temp
    else:
      robot.left_motor.value = speed_value
      robot.right_motor.value = angle_temp
      
    
execute({'new': camera.value})
camera.observe(execute, names='value')

object : 0, left_motor : 0.0, right_motor : -2.0129894801993284
object : 0, left_motor : 0.2661047968419968, right_motor : 2.06906568842409
object : 0, left_motor : 0.2675059203051101, right_motor : 2.0298342314569164
object : 0, left_motor : 0.26732979150241387, right_motor : 2.034765837932411
object : 0, left_motor : 0.2684392958564491, right_motor : 2.0036997160194248
object : 0, left_motor : 0.266956829541244, right_motor : 2.0452087728451684
object : 0, left_motor : 0.26660482755243636, right_motor : 2.0550648285317825
object : 0, left_motor : 0.2669751684236802, right_motor : 2.0446952841369552
object : 0, left_motor : 0.26551630326530024, right_motor : 2.0855435085715937
object : 0, left_motor : 0.2668539823602303, right_motor : 2.048088493913553
object : 0, left_motor : 0.26660916880463875, right_motor : 2.054943273470115
object : 0, left_motor : 0.26707373319113253, right_motor : 2.0419354706482893
object : 0, left_motor : 0.26548745657344086, right_motor : 2.0863512159436564


### Execution STOP

In [23]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

object : 0, left_motor : 0.2684451044387417, right_motor : 2.0035370757152338
object : 0, left_motor : 0.0, right_motor : 0.0


###### 